# 03_path_length_sensitivity.ipynb

**Claim under test:**  
Pathwise regret scales ~ linearly with P_T at fixed λ and G.

**Grid file:** `grids/03_path_length_sensitivity.yaml`

**Baseline & conventions**
- Use the STAR schema built by `experiment/utils/duck-db-loader.py`.
- Slice results by `(grid_id, seed)` using `analytics.v_run_summary`.
- Keep plots identical across notebooks; only the *dial* under test changes.
- Reference path-length/regret-decomposition companion analysis for context. :contentReference[oaicite:0]{index=0}

## 0) Imports & paths

In [1]:
from pathlib import Path
import duckdb
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

# go up one level
os.chdir("..")

# project paths (edit if your repo layout differs)
REPO = Path(".").resolve()
RESULTS_GLOB = "results_parquet/03_path_length_sensitivity/events/grid_id=*/seed=*/*.parquet"  # corrected pattern
DB_PATH = REPO/"artifacts"/"star.duckdb"
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

GRID_FILE = REPO/"grids/03_path_length_sensitivity.yaml"     # this notebook’s grid
STAGING_TABLE = "staging.events"   # keep default from loader

## 1) Build/load the STAR schema

In [2]:
from utils.duck_db_loader import load_star_schema

con = load_star_schema(
    input_path=str(RESULTS_GLOB),
    db_path=str(DB_PATH),
    staging_table=STAGING_TABLE,
    run_ddl=True,
    create_events_view=True,
)

# sanity: counts
display(con.execute("""
SELECT 'dim_run' t, COUNT(*) c FROM analytics.dim_run
UNION ALL SELECT 'fact_event', COUNT(*) FROM analytics.fact_event
UNION ALL SELECT 'dim_event_type', COUNT(*) FROM analytics.dim_event_type
""").df())

IOException: IO Error: Could not set lock on file "/workspaces/unlearning-research-meta/experiment/artifacts/star.duckdb": Conflicting lock is held in /usr/local/bin/python3.11 (PID 21902). See also https://duckdb.org/docs/stable/connect/concurrency

## 2) Snapshot config & runs

In [ ]:
# show the grid dictionary if kept in YAML (optional)
try:
    import yaml, textwrap
    cfg = yaml.safe_load(Path(GRID_FILE).read_text())
    print("grid file:", GRID_FILE)
    print(json.dumps(cfg.get("matrix", {}), indent=2))
except Exception as e:
    print("Note: could not parse YAML grid:", e)

runs = con.execute("""
SELECT *
FROM analytics.v_run_summary
ORDER BY grid_id, seed
""").df()
runs.head()

## 3) Standard query helpers

In [ ]:
def last_event_frame(con, grid_id:str, seed:int):
    return con.execute("""
    WITH last AS (
      SELECT MAX(event_id) AS last_id
      FROM analytics.fact_event fe
      WHERE fe.grid_id = ? AND fe.seed = ?
    )
    SELECT fe.*
    FROM analytics.fact_event fe
    CROSS JOIN last l
    WHERE fe.event_id = l.last_id AND fe.grid_id = ? AND fe.seed = ?
    """, [grid_id, seed, grid_id, seed]).df()

def trace_frame(con, grid_id:str, seed:int, cols:tuple[str,...]):
    col_list = ", ".join(cols)
    return con.execute(f"""
    SELECT event_id, {col_list}
    FROM analytics.fact_event fe
    WHERE fe.grid_id = ? AND fe.seed = ?
    ORDER BY event_id
    """, [grid_id, seed]).df()

## 4) Plots (standardized)
Keep styling consistent across notebooks for visual comparability.

In [ ]:
def plot_traces(df, ycols, title):
    for y in ycols:
        plt.figure()
        plt.plot(df["event_id"], df[y], label=y)
        plt.xlabel("event_id"); plt.ylabel(y); plt.title(f"{title}: {y}")
        plt.legend(); plt.show()

## 5) Notebook-specific check: claim, dial, metrics

In [ ]:
NOTEBOOK_CLAIM = "Pathwise regret scales ~ linearly with P_T at fixed λ and G."
print("Claim under test:", NOTEBOOK_CLAIM)

# choose a (grid_id, seed) to visualize (edit if multiple)
if not runs.empty:
    gid, seed = runs.loc[0, ["grid_id","seed"]]
    print("Example run:", gid, seed)

    # universal traces
    df = trace_frame(con, gid, int(seed), (
        "cum_regret", "regret_static_term", "regret_path_term",
        "P_T_true", "ST_running", "g_norm", "eta_t"
    ))
    plot_traces(df, ["cum_regret","regret_static_term","regret_path_term"], "Regret decomposition")
    plot_traces(df, ["P_T_true","ST_running"], "Path & energy")
    plot_traces(df, ["g_norm","eta_t"], "Grad norm & step size")

    # end-of-run snapshot
    tail = last_event_frame(con, gid, int(seed))
    display(tail[["cum_regret","cum_regret_with_noise","P_T_true","ST_running","rho_spent","m_used"]])

## 5.5) Standardized Analysis Follow-ups
Execute common analyses across all experiment notebooks for automated claim checks and theory validation.

In [ ]:
import numpy as np
# Import standardized analysis functions
import sys
sys.path.append('../utils')
from standardized_analysis import run_all_standardized_analyses, enhance_claim_check_export

# Run all standardized analyses
analysis_results = run_all_standardized_analyses(con, runs)

# Display summary statistics
print("\n=== THEORY BOUND TRACKING ===")
theory_results = analysis_results['theory_bound_tracking']
successful_theory = [r for r in theory_results.values() if r['status'] == 'success']
if successful_theory:
    ratios = [r['theory_ratio_final'] for r in successful_theory if r['theory_ratio_final'] is not None]
    if ratios:
        print(f"Theory ratio - Mean: {np.mean(ratios):.3f}, Median: {np.median(ratios):.3f}")
        print(f"Expected: O(1) when theory matches experiment")
        
print("\n=== STEPSIZE POLICY VALIDATION ===")
stepsize_results = analysis_results['stepsize_policy_validation']
policy_pass = sum(1 for r in stepsize_results.values() if r['stepsize_policy_status'] == 'pass')
policy_total = len([r for r in stepsize_results.values() if r['stepsize_policy_status'] in ['pass', 'fail']])
if policy_total > 0:
    print(f"Stepsize policy adherence: {policy_pass}/{policy_total} runs passed")
    
print("\n=== PRIVACY & ODOMETER SANITY CHECKS ===")
privacy_results = analysis_results['privacy_odometer_checks']
privacy_pass = sum(1 for r in privacy_results.values() if r['privacy_odometer_status'] == 'pass')
privacy_total = len([r for r in privacy_results.values() if r['privacy_odometer_status'] in ['pass', 'fail']])
if privacy_total > 0:
    print(f"Privacy/Odometer checks: {privacy_pass}/{privacy_total} runs passed")
    
print("\n=== SEED STABILITY AUDIT ===")
stability_results = analysis_results['seed_stability_audit']
flagged_grids = [grid for grid, stats in stability_results.items() 
                if stats.get('high_variability_flag', False)]
total_grids = len([s for s in stability_results.values() if s['status'] == 'success'])
print(f"High variability grids: {len(flagged_grids)}/{total_grids}")
if flagged_grids:
    print(f"Flagged grids: {flagged_grids}")

## 6) One-page “claim check” (export)
Emits a compact JSON summary to artifacts/ for CI diffs.

In [ ]:
ARTIFACT = REPO/"artifacts"/(Path("03_path_length_sensitivity.ipynb").stem + "_claim_check.json")
ARTIFACT.parent.mkdir(parents=True, exist_ok=True)

# Get base summary
summary = con.execute("""
SELECT
  dr.grid_id, dr.seed,
  MAX(fe.cum_regret) FILTER (WHERE fe.event_id = (SELECT MAX(event_id) FROM analytics.fact_event WHERE grid_id = fe.grid_id AND seed = fe.seed)) AS final_cum_regret,
  MAX(fe.cum_regret_with_noise) FILTER (WHERE fe.event_id = (SELECT MAX(event_id) FROM analytics.fact_event WHERE grid_id = fe.grid_id AND seed = fe.seed)) AS final_cum_regret_with_noise,
  MAX(fe.P_T_true) FILTER (WHERE fe.event_id = (SELECT MAX(event_id) FROM analytics.fact_event WHERE grid_id = fe.grid_id AND seed = fe.seed)) AS final_P_T_true,
  MAX(fe.ST_running) FILTER (WHERE fe.event_id = (SELECT MAX(event_id) FROM analytics.fact_event WHERE grid_id = fe.grid_id AND seed = fe.seed)) AS final_ST_running,
  MAX(fe.rho_spent) FILTER (WHERE fe.event_id = (SELECT MAX(event_id) FROM analytics.fact_event WHERE grid_id = fe.grid_id AND seed = fe.seed)) AS final_rho_spent,
  MAX(fe.m_used) FILTER (WHERE fe.event_id = (SELECT MAX(event_id) FROM analytics.fact_event WHERE grid_id = fe.grid_id AND seed = fe.seed)) AS final_m_used
FROM analytics.fact_event fe
JOIN analytics.dim_run dr USING (grid_id, seed)
GROUP BY dr.grid_id, dr.seed
ORDER BY dr.grid_id, dr.seed
""").df().to_dict(orient="records")

# Enhance summary with standardized analysis results
enhanced_summary = enhance_claim_check_export(
    summary, 
    analysis_results["theory_bound_tracking"],
    analysis_results["stepsize_policy_validation"],
    analysis_results["privacy_odometer_checks"],
    analysis_results["seed_stability_audit"]
)

ARTIFACT.write_text(json.dumps({
    "notebook": "03_path_length_sensitivity.ipynb",
    "claim": NOTEBOOK_CLAIM,
    "grid_file": str(GRID_FILE),
    "summary": enhanced_summary,
    "standardized_analyses": {
        "theory_bound_tracking": analysis_results["theory_bound_tracking"],
        "stepsize_policy_validation": analysis_results["stepsize_policy_validation"],
        "privacy_odometer_checks": analysis_results["privacy_odometer_checks"],
        "seed_stability_audit": analysis_results["seed_stability_audit"]
    }
}, indent=2))
print("Wrote:", ARTIFACT)